# This Notebook presents an expriment of Theory Of Mind on Rock Paper Scissors
### The goal is to research and devlop a method for generic multi-agent games using Theory of Mind modeling.
### We will introduce the tradeoff between choosing optimal move and staying unpredictable.

In [1]:
# Import libraries.
using Turing, StatsPlots, Random

### computer_counter_policy
Given infered information about the opponent.<br>
Decide how to exploit it.<br>
In the future we will optimize those parameters, to make the agent's move less predictible <br>

different counter policy computing.
not optimal! will be changed

given history we want to know the distributin of the moves

## The core of the notebook
### Modeling Theory of Mind in Rock Paper Sciors

## The model of agent as universal Probalistic Model
### gets some parmeters:
<ul>
    <li> opponent agent - representation of our belief on the opponent model (Turing.jl model)
    <li> my history - list of all moves this player done (At this point unused)
    <li> opponent history - list of all moves the opponent done, used to estimate a prior on the moves
    <li> depth - the depth we want the agent will dive modeling the mind of opponent agent
</ul>        

In [2]:
function best_move(opp_move)
    best_moves = Dict(1 => 2, 2 => 3, 3 => 1)
    move = best_moves[opp_move]
    println(move)
#     p_deceive = 0.001
#     print(p_deceive)
#     deceive ~ Bernoulli(p_deceive)
#     print(deceive)
#     if deceive
#         move = opp_move
#     end
    return move
end

best_move (generic function with 1 method)

In [3]:
@model function agent(opponent_agent, my_history, opponent_history, depth = 1, discrete_sampler = PG, discrete_sampler_hyper_param=1, num_of_iterations=1)
    if depth == 0
#         println("in depth 0")
        beta_opp ~ Dirichlet([1, 1, 1])
#         println("length is ", length(opponent_history))
        for opp_move in opponent_history
            opp_move ~ Categorical(beta_opp) # observe
        end
    next_opp_move ~ Categorical(beta_opp) # sample
#     println("next opp move is  ", next_opp_move)
    else
        opponent_model = opponent_agent(agent, opponent_history, my_history, depth-1)
#         print("before sampling")
        next_opp_move_chain  = sample(opponent_model, discrete_sampler(discrete_sampler_hyper_param), num_of_iterations, progress=true); # if you use MH - you need to burn samples in the chain first
        result = generated_quantities(opponent_model, next_opp_move_chain)
#         print("result is " , display(result))
        # last sample
#         println("chain is ready and it is ", display(next_opp_move_chain))
#         println("result is ready and it is ", result)
        next_opp_move_from_chain = last(result)
        next_opp_move ~ Dirac(next_opp_move_from_chain)
#         println("next opp is " ,next_opp_move)
    end
    move = best_move(next_opp_move)
    println("move is ", move)
    return move
end

agent (generic function with 5 methods)

the infece is about the opponent model and not my model

In [4]:
m = agent(agent, ones(Int64, 10) , 2* ones(Int64, 10))
chain = sample(m, PG(3), 3)
display(chain)
return_values = generated_quantities(m, chain)

3
move is 3
3
move is 3
2
move is 2
3
move is 3
1
move is 1
1
move is 1
3
move is 3
3
move is 3
3
move is 3
1
move is 1
2
move is 2
1
move is 1
3
move is 3
1
move is 1
2
move is 2
3
move is 3
1
move is 1
3
move is 3
1
move is 1
2
move is 2
2
move is 2
1
move is 1
2
move is 2
1
move is 1
2
move is 2
1
move is 1
2
move is 2
3
move is 3
2
move is 2
3
move is 3
2
move is 2
3
move is 3
2
move is 2
3
move is 3
1
move is 1


Sampling:  67%|████████████████████████████             |  ETA: 0:00:00

2
move is 2


Sampling: 100%|█████████████████████████████████████████| Time: 0:00:00


Chains MCMC chain (3×3×1 Array{Float64,3}):

Log evidence      = 0.0
Iterations        = 1:3
Thinning interval = 1
Chains            = 1
Samples per chain = 3
parameters        = next_opp_move
internals         = logevidence, lp

Summary Statistics
     parameters      mean       std   naive_se      mcse       ess      rhat 
         Symbol   Float64   Float64    Float64   Missing   Missing   Missing 

  next_opp_move    2.3333    1.1547     0.6667   missing   missing   missing

Quantiles
     parameters      2.5%     25.0%     50.0%     75.0%     97.5% 
         Symbol   Float64   Float64   Float64   Float64   Float64 

  next_opp_move    1.1000    2.0000    3.0000    3.0000    3.0000


3
move is 3
3
move is 3
3
move is 3
1
move is 1
2
move is 2
2
move is 2
3
move is 3
2
move is 2
3
move is 3
1
move is 1
1
move is 1
2
move is 2
1
move is 1
2
move is 2
1
move is 1
3
move is 3
2
move is 2
3
move is 3
1
move is 1
1
move is 1
2
move is 2
3
move is 3
2
move is 2
2
move is 2
2
move is 2
2
move is 2
3
move is 3
2
move is 2
1
move is 1
1
move is 1
1
move is 1
2
move is 2
1
move is 1
1
move is 1


3×1 Array{Int64,2}:
 2
 1
 1

### simple aux function that return the most common sample - majority vote

In [61]:
function most_common(samples)
    count = Dict(1 => 0, 2 => 0, 3 => 0)
    for i in 1:length(samples)
        count[samples[i]] += 1
    end
    max_k, max_v = -1 , -1
    for (k, v) in count
        if v > max_v
            max_k , max_v = k, v
        end
    end
    return max_k
end

most_common (generic function with 1 method)

### This method is used for the simulation, each model make a move at the end of this function, as a result of reasoning of the model

In [62]:
function move(agent, other_agent, my_history, other_agent_history, my_depth=1)
    other_agent_history = length(other_agent_history) > 0 ? other_agent_history : [1]
    my_history = length(my_history) > 0 ? my_history : [1]
    my_history = Array{Int}(my_history)
    other_agent_history = Array{Int}(other_agent_history)
    chain = sample(agent(other_agent, my_history, other_agent_history, my_depth), PG(5), 5, progress = true)
    return most_common(chain[:"next_move"])
end

move (generic function with 2 methods)

In [63]:
move(agent, agent, [1, 2], [2 , 3])
move(agent, agent, [1, 2, 3], [2 , 3, 2])
move(agent, agent, [1, 2, 2, 1], [2 , 3, 2, 3])

Sampling: 100%|█████████████████████████████████████████| Time: 0:00:00
Sampling: 100%|█████████████████████████████████████████| Time: 0:00:00


3

### Game simulation , given two agent models , depth params, let them play num of simulation times

In [64]:
function game(first_player_depth = 1, second_player_depth = 1)
    first_player = agent
    second_player = agent
    num_of_simulations = 100
    first_player_history = []
    second_player_history = []
    for i in 1:num_of_simulations
        m1 = move(first_player, second_player, first_player_history, second_player_history, first_player_depth)
#         println("player1 choose $m1")
        push!(first_player_history, m1)
        m2 = move(second_player, first_player, second_player_history, first_player_history, second_player_depth)
#         println("player2 choose $m2")
        push!(second_player_history, m2)
#         println("in simulation $i first player chose $m1 second player chose $m2")
    end
    return first_player_history, second_player_history
 end

game (generic function with 3 methods)

In [65]:
function score(history)
    first_player_history, second_player_history = history
    first_wins = 0
    ties = 0
    second_wins = 0
    wins = Dict(1 => 3, 2 => 1, 3 => 2)
    for i in 1:length(first_player_history)
        if wins[first_player_history[i]] == second_player_history[i]
            first_wins += 1
        elseif wins[second_player_history[i]] == first_player_history[i]
            second_wins += 1
        else
            ties += 1
        end
    end
    return first_wins, ties, second_wins
end

score (generic function with 1 method)

In [66]:
function display_score(score)
    num_of_wins_first, num_of_ties, num_of_wins_second = score
    println("first player won: $num_of_wins_first")
    println("second player won: $num_of_wins_second") 
    println("ties: $num_of_ties") 
end

display_score (generic function with 1 method)

both player plays without theory of mind - equivilent

In [67]:
display_score(score(game(0, 0)))

first player won: 14
second player won: 11
ties: 75


2nd player plays with one level of theory of mind -> 2nd wins

In [68]:
display_score(score(game(0, 1)))

Sampling: 100%|█████████████████████████████████████████| Time: 0:00:00
Sampling: 100%|█████████████████████████████████████████| Time: 0:00:00
Sampling: 100%|█████████████████████████████████████████| Time: 0:00:00
Sampling: 100%|█████████████████████████████████████████| Time: 0:00:00
Sampling: 100%|█████████████████████████████████████████| Time: 0:00:00
Sampling: 100%|█████████████████████████████████████████| Time: 0:00:00
Sampling: 100%|█████████████████████████████████████████| Time: 0:00:00
Sampling: 100%|█████████████████████████████████████████| Time: 0:00:00
Sampling: 100%|█████████████████████████████████████████| Time: 0:00:00
Sampling: 100%|█████████████████████████████████████████| Time: 0:00:00
Sampling: 100%|█████████████████████████████████████████| Time: 0:00:00
Sampling: 100%|█████████████████████████████████████████| Time: 0:00:00
Sampling: 100%|█████████████████████████████████████████| Time: 0:00:00
Sampling: 100%|█████████████████████████████████████████| Time: 

first player won: 16
second player won: 28
ties: 56


1st player plays with one level of theory of mind -> 1st wins

In [69]:
display_score(score(game(1, 0)))

Sampling: 100%|█████████████████████████████████████████| Time: 0:00:00
Sampling: 100%|█████████████████████████████████████████| Time: 0:00:00
Sampling: 100%|█████████████████████████████████████████| Time: 0:00:00


first player won: 32
second player won: 7
ties: 61


both players play with one level of theory of mind -> preety much equivelent

In [70]:
display_score(score(game(1, 1)))

Sampling: 100%|█████████████████████████████████████████| Time: 0:00:00


first player won: 24
second player won: 23
ties: 53
